In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import string

from clean_data import*
from add_summary import*
from find_category import*
from get_popularity import*
from get_sentiment import*

In [ ]:
## the following tables appear in order according to the functions in model.py

In [2]:
## source table: newspaper information
source=DataFrame(columns=["id", "name", "URL_logo"])

source1=pd.DataFrame([[1, "CNN", "http://www.logospike.com/wp-content/uploads/2014/11/Cnn_logo-2.png"],
                      [2, "FoxNews", "https://upload.wikimedia.org/wikipedia/commons/d/d4/Fox_News_Channel_logo.png"],
                      [3, "New York Times", "http://1000logos.net/wp-content/uploads/2017/04/Logo-New-York-Times.jpg"]],columns=["id", "name", "URL_logo"])

source=source.append(source1)


## state table: state information
state=DataFrame(columns=["id", "name", "URL_logo","primaries_date"])

state1=pd.DataFrame([[1, "West Virginia", "https://www.united-states-flag.com/media/catalog/product/cache/2/image/9df78eab33525d08d6e5fb8d27136e95/w/e/westvirginia-nylon_1.jpg", "May 8 2018"],
                     [2, "Virginia", "https://www.50states.com/images/redesign/flags/va-largeflag.png", "June 12 2018"],
                     [3, "Texas", "https://en.wikipedia.org/wiki/Flag_of_Texas#/media/File:Flag_of_Texas.svg", "NA"]], 
                    columns=["id", "name", "URL_logo","primaries_date" ])

state=state.append(state1)


## issue table: main issues and a descriptions

issue=DataFrame(columns=["id", "name","info","URL_logo"])

issue1=pd.DataFrame([[1, "Gun Law", "Gun laws in the United States regulate the sale, possession, and use of firearms and ammunition. State laws (and the laws of Washington, D.C. and the U.S. territories) vary considerably, and are independent of existing federal firearms laws, although they are sometimes broader or more limited in scope than the federal laws. State level laws vary significantly in their form, content, and level of restriction. Forty-four states have a provision in their state constitutions similar to the Second Amendment to the U.S. Constitution, which protects the right to keep and bear arms. The exceptions are California, Iowa, Maryland, Minnesota, New Jersey, and New York. In New York, however, the statutory civil rights laws contain a provision virtually identical to the Second Amendment. Additionally, the U.S. Supreme Court held in McDonald v. Chicago that the protections of the Second Amendment to keep and bear arms for self-defense in one's home apply against state governments and their political subdivisions.", "https://cdn.cltampa.com/files/base/scomm/cltampa/image/2018/03/640w/gun_law.5aa311dbc246a.jpg"],
                     [2, "Immigration","Immigration is the international movement of people into a destination country of which they are not natives or where they do not possess citizenship in order to settle or reside there, especially as permanent residents or naturalized citizens, or to take-up employment as a migrant worker or temporarily as a foreign worker.As for economic effects, research suggests that migration is beneficial both to the receiving and sending countries. Research, with few exceptions, finds that immigration on average has positive economic effects on the native population, but is mixed as to whether low-skilled immigration adversely affects low-skilled natives. Studies show that the elimination of barriers to migration would have profound effects on world GDP, with estimates of gains ranging between 67 and 147 percent. Development economists argue that reducing barriers to labor mobility between developing countries and developed countries would be one of the most efficient tools of poverty reduction." ,"http://toledocitypaper.com/wp-content/uploads/2017/03/immigrants-toledo-refugees-protest.jpg"],
                     [3, "Foreign Policy", "A country's foreign policy, also called foreign relations or foreign affairs policy, consists of self-interest strategies chosen by the state to safeguard its national interests and to achieve goals within its international relations milieu. The approaches are strategically employed to interact with other countries. The study of such strategies is called foreign policy analysis. In recent times, due to the deepening level of globalization and transnational activities, the states will also have to interact with non-state actors. The aforementioned interaction is evaluated and monitored in attempts to maximize benefits of multilateral international cooperation. Since the national interests are paramount, foreign policies are designed by the government through high-level decision making processes. National interests accomplishment can occur as a result of peaceful cooperation with other nations, or through exploitation. Usually, creating foreign policy is the job of the head of government and the foreign minister (or equivalent).", "https://bacon.house.gov/sites/bacon.house.gov/files/styles/congress_featured_image/public/featured_image/issues/Foreign%20Policy.png?itok=UVYCCIOO"],
                     [4, "Economy", "An economy is an area of the production, distribution, or trade[1], and consumption of goods and services by different agents. Understood in its broadest sense, 'The economy is defined as a social domain that emphasizes the practices, discourses, and material expressions associated with the production, use, and management of resources'.[2] Economic agents can be individuals, businesses, organizations, or governments. Economic transactions occur when two parties agree to the value or price of the transacted good or service, commonly expressed in a certain currency. However, monetary transactions only account for a small part of the economic domain. Economic activity is spurred by production which uses natural resources, labor, and capital. It has changed over time due to technology (automation, accelerator of process, reduction of cost functions), innovation (new products, services, processes, new markets, expanding markets, diversification of markets, niche markets, increases revenue functions) such as, that which produces intellectual property and changes in industrial relations (for example, child labor being replaced in some parts of the world with universal access to education).", "http://media.al.com/news_impact/photo/economy-2014jpg-886912a5cd9c9100.jpg"],
                     [5, "Crime", "In ordinary language, a crime is an unlawful act punishable by a state or other authority. The term crime does not, in modern criminal law, have any simple and universally accepted definition, though statutory definitions have been provided for certain purposes.The most popular view is that crime is a category created by law; in other words, something is a crime if declared as such by the relevant and applicable law.One proposed definition is that a crime or offence (or criminal offence) is an act harmful not only to some individual but also to a community, society or the state (a public wrong). Such acts are forbidden and punishable by law.", "https://www.brennancenter.org/sites/default/files/styles/individual_node_page/public/blog/crime%20cuffs.jpg?itok=WP0o5xht"]] ,
                    columns=["id", "name", "info","URL_logo" ])


issue=issue.append(issue1)


## article table: TBD
# FK: candidate, issue, state, source
# link, summary, date (datefield)
# sentiment (int), 

## candidate table: name, biographical details, etc.
candidate=DataFrame(columns=["id", "name","state_fk","state","date_of_birth", "place_birth", "position", "URL_photo", "party","first_name","last_name"])

candidate1=pd.DataFrame([[1, "Don Blankenship",1,"West_Virginia", "1950-03-14", "Stopover Kentucky", "Coal magnate", "https://i2.wp.com/woay.tv/wp-content/uploads/2017/05/GTY_don_blankenship_jt_160406_4x3_992.jpg?resize=735%2C400&ssl=1","r","Don", "Blankenship"],
                     [2, "Bo Copley",1,"West_Virginia", "1900-01-01", "Dingess West Virginia", "Coal Miner","https://heavyeditorial.files.wordpress.com/2017/05/bo-copley-e1494012597984.jpg?quality=65&strip=all&w=780&strip=all","r","Bo", "Copley"],
                     [3, "Evan Jenkins",1,"West_Virginia", "1960-09-12", "Huntington West Virginia","U.S. House of Representatives","https://www.congress.gov/img/member/114_rp_wv_3_jenkins_evan.jpg","r","Evan", "Jenkins"],
                     [4, "Joe Manchin",1,"West_Virginia", "1947-08-24", "Farmington West Virginia", "United States Senator", "https://fi.wikipedia.org/wiki/Joe_Manchin#/media/File:Senator_Manchin.jpg", "d","Joe", "Manchin"],
                     [5, "Patrick Morrisey",1,"West_Virginia","1967-12-21","Brooklyn New York City NY","West Virginia Attorney General since 2013","http://wvmetronews.com/wordpress/wp-content/uploads/2016/02/Patrick-Morrisey.jpg?x43308","r","Patrick", "Morrisey"],
                     [6, "Jack Newbrough",1,"West_Virginia", "1900-01-01", "U.S.","U.S. Navy veteran","http://s3.amazonaws.com/ogden_images/www.weirtondailytimes.com/images/2017/09/23190845/Newbrough-003-1100x825.jpg" ,"r","Jack", "Newbrough"],
                     [7, "Paula Jean Swearengin", 1,"West_Virginia", "1973-01-01","Mullens West Virginia","Activist","https://cbbba9d899c5f8ae6108-e379174d4d25dcb78174c6cab0a88cea.ssl.cf1.rackcdn.com/1275564-l.jpg", "d","Paula Jean", "Swearengin"],
                     [8, "Tom Willis",1,"West_Virginia", "1900-01-01","U.S.","Businessman","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/2C512069-7298-4BAC-B9F3-6BFECDC2FB57.jpeg","r","Tom" ,"Willis"],
                     [9, "Nicholas Freitas", 2,"Virginia", "1979-08-29", "Chico California", "Member of the Virginia House of Delegates","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/Nick_Freitas.jpg","r","Nicholas", "Freitas"],
                     [10, "E. W. Jackson", 2,"Virginia","1952-01-13","Chester Pennsylvania","Lawyer","http://www.standardnewswire.com/images/1248274804.jpg","r","E.W.", "Jackson"],
                     [11, "Tim Kaine", 2,"Virginia", "1958-02-26", "Saint Paul Minnesota", "United States Senator", "https://upload.wikimedia.org/wikipedia/commons/1/1d/Tim_Kaine%2C_official_113th_Congress_photo_portrait.jpg", "d","Tim", "Kaine"],
                     [12, "Corey Stewart", 2,"Virginia", "1968-08-01", "Duluth Minnesota", "Lawyer", "http://www.pwcgov.org/government/bocs/PublishingImages/2012%20Official%20Portrait%20Chairman%20Stewart.jpg", "r","Corey", "Stewart"],
                     [13, "Matt Waters", 2,"Virginia", "1900-01-01", "Newport News Virginia", "Director of Development at Students For Liberty", "https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/Matt_Waters.jpg", "o","Matt", "Waters"],
                     [14, "Carl Bible",3, "Texas", "1900-01-01", "Texas", "Businessman", "https://pbs.twimg.com/profile_images/928089633296023552/Vm-6G8IN_400x400.jpg", "i","Carl", "Bible"],
                     [15, "Ted Cruz", 3,"Texas", "1970-12-22", "Calgary Alberta Canada", "United States Senator", "https://upload.wikimedia.org/wikipedia/commons/8/87/Ted_Cruz%2C_official_portrait%2C_113th_Congress.jpg", "r","Ted", "Cruz"],
                     [16, "Bob McNeil", 3,"Texas", "1949-01-23", "Stuttgart Arkansas", "Businessman", "https://static1.squarespace.com/static/5987cd8617bffc6a2723b230/t/59b20ddff14aa1ef98ae1083/1508991634672/20170804+-+RAM+Portrait-Cropped-100dpi.JPG?format=300w", "o","Bob", "McNeil"],
                     [17, "Beto ORourke", 3,"Texas", "1972-09-26", "El Paso Texas", "U.S. House of Representatives", "https://upload.wikimedia.org/wikipedia/commons/a/ad/Beto_O%27Rourke%2C_Official_portrait%2C_113th_Congress.jpg", "d","Beto", "ORourke"]],
                        columns=["id", "name","state_fk","state","date_of_birth", "place_birth", "position", "URL_photo", "party","first_name","last_name"])

candidate=candidate.append(candidate1)

candidate["score_issue_1"]=0.5
candidate["score_issue_2"]=0.5
candidate["score_issue_3"]=0.5
candidate["score_issue_4"]=0.5
candidate["score_issue_5"]=0.5

In [3]:
all_candidates = []
for index,row in candidate.iterrows():
    all_candidates.append(row['name'])
#all_candidates.extend(["EW Jackson", "Beto "])

In [4]:
folder = "RAW_DATA/"
state_data_westvirginia = ["cnn_westvirginia.json","jsfoxnews_westvirginia.json","nyt_westvirginia.json"]
state_data_virginia = ["cnn_virginia.json","jsfoxnews_virginia.json","nyt_westvirginia.json"]
state_data_texas = ["cnn_texas.json","jsfoxnews_texas.json","nyt_texas.json"]
candidate_data_missing = ["cnn_general.json", "jsfoxnews_general.json", "nyt_general.json"]

#add in new data later...
all_state_data = [state_data_westvirginia, state_data_virginia, state_data_texas, candidate_data_missing]


In [5]:
## script is hardcoded to take in data from the RAW_DATA file
# this function takes in a list of candidates (string form) and a name of the output json file
structure_data(folder, all_state_data, all_candidates, candidate, source, "scraped_data_mixed.json")

In [6]:
key_words = ["festival", "theater", "music", "concert", "dance", "performance"
                       , "jazz", "band", "arts", "art","museum","painting", "artist", "exhibition","gallery", "sculpture"
                       ,"album", "film", "movie", "opera", "cookbook", "orchestra"
                       ,"design", "photograph", "drawing", "design"
                       , "production","quartet", "series", "star", "feature"
                       ,"village", "production", "season", "premiere","ballet", "photograph", "drawing", "boat"
                       , "symphony", "shakespeare", "repertory","pop", "perform", "classical", "musical"
                       "literature", "westworld", "obituary", "episode", "athlete"]
remove_irrelevant_articles("scraped_data_mixed.json", key_words, "scraped_data_0.json")

In [ ]:
''' weird articles
Hidden volcanoes melt Antarctic glaciers from below, study finds
Hockey Bus Crash Victims Include Respected Coach, Captain 3
Today in History 3
Corrections: April 10, 2018 3
How to Stay on Top of Breaking News 3
WHERE MERRY TONGUES TATTLE; What Wet the Wireless.
61 ATHLETES IN COMPETITION.; ANNUAL GAMES OF THE MONTGOMERY CLUB OF JERSEY CITY.
Obituary 1 -- No Title
2 for 22: McCutchen Walks Off SF Again, This Time vs D-Backs
65 EDMUND C. ALBERT--ELIZABETH H. RING.
The Listings: Dec. 30 - Jan. 5
ON THE ROAD; A City of Surprises, Not All Big
Patriots End a Week of Hand-Wringing With Another A.F.C. Crown
‘The Assassination of Gianni Versace’ Episode 3: Death or Disgrace?
77 DealBook Briefing: Blue Apron’s Co-Founder Steps Down as C.E.O.
78 DealBook Briefing: Pass-Throughs vs. Corporations — Who’s the Bigger Winner?
80 Gossip From the Liz Smith Memorial, Just the Way She’d Want It
The Latest: Suspect to be charged with capital murder
NFL Network analyst suggests Eagles put bounty on Ezekiel Elliott in bizarre rant
Obituary 1 -- No Title 6
333 Today in History 14
392 Washington knows that 🏀 is life 11
416 Who is Sam Nunberg? 15

397 West Virginia Senate hopefuls take center stage ahead of Fox News debate"], "articles_text": ["MORGANTOWN, W.Va. –  Sen. Mitch McConnell, R-Ky. is the leader of the Senate. But one Republican primary candidate is talking about the longtime Kentucky lawmaker like he’s the leader of a drug cartel. “One of my goals as U.S. Senator will be to ditch cocaine Mitch,” candidate Don Blankenship declared at the end of a new campaign ad. The origins of “cocaine Mitch,” according to the campaign, are rooted in accusations of drug smuggling on Chinese ships associated with the family of McConnell’s wife, Transportation Secretary Elaine Chao. The narcotic-themed nickname enters the dialogue on the eve of a nationally televised debate at 6:30 p.m. Tuesday on Fox News Channel. Blankenship will be joined on stage in Morgantown by Rep. Evan Jenkins, R-W.Va. and West Virginia Attorney General Patrick Morrisey. Considered a toss-up in the midterm elections, the West Virginia Senate race is one where Republicans hope to have the chance to unseat incumbent Joe Manchin, a Democrat who first won the seat in a 2010 special election. According to a Fox News poll, three Republicans are considered to be “top tier” candidates ahead of the May 8 primary: Blankenship, Evan Jenkins and Patrick Morrisey. Jenkins, a U.S. congressman, topped the list of Republican contenders. But the race is getting national attention because a group with ties to GOP leadership, the Mountain Families PAC, has spent at least $1.3 million to try to derail Blankenship’s candidacy. Operatives fear the year in prison the former coal baron recently served on a conspiracy to violate mine safety standards, tied to the 2010 Upper Big Branch mine disaster that killed 29, may jeopardize an opportunity to unseat Manchin. Blankenship shrugs off the impact of the intra-party effort to keep him off the November ballot. “I think it probably moves me up a little bit,” Blankenship said of the ads blanketing TV and radio broadcasts, calling him “convicted criminal Don Blankenship.”  “I don’t think Mitch McConnell realizes,” he said, “how anti-him and how anti-the establishment West Virginians are.” While Blankenship's bid is a long shot, he's testing whether a party led by an anti-establishment outsider can rein in its anti-establishment impulses. "The establishment, no matter who you define it as, has not been creating jobs in West Virginia," Blankenship said at a primary debate this past week. Jenkins, who leads the field of Republicans, has been a congressman for three years and served as a state senator in West Virginia for 12 years. Morrisey is the first Republican to serve as attorney general in the state since 1933, according to his campaign website. Manchin is not running unopposed; he races a primary challenger in Paula Jean Swearengin, an environmental activist whose platform is more in line with Sen. Bernie Sanders, the progressive independent from Vermont. President Trump beat Democrat Hillary Clinton by more than 40 points in the Mountain State in the 2016 presidential election. The state has long been considered a prime pickup opportunity for Republicans, who hold a two-seat Senate majority that suddenly feels less secure given signs of Democratic momentum in Nevada, Arizona, Tennessee and elsewhere. If Democrats can win West Virginia, which gave Trump his largest margin of victory in the nation, they may have a slim chance at seizing the Senate majority. 7
'''

In [7]:
add_category("set_2_data/testfile_tagged.txt", "scraped_data_0.json", "scraped_data_1_cat.json")

In [4]:
## add manual articles to scraped data

manual_articles = pd.read_json("extra_articles.json")
scraped_articles = pd.read_json("scraped_data_1_cat.json")

scraped_articles = scraped_articles.append(manual_articles)
scraped_articles = reorder_df(scraped_articles)


with open("ALL_DATA_2.json", 'w') as f:
    f.write(scraped_articles.to_json(orient = "records"))

In [6]:
# this script takes an input file and creates a new json
add_summary("ALL_DATA_2.json", "ALL_DATA_SUMM_3.json")

In [8]:
get_sentiment("ALL_DATA_SUMM_3.json", "ALL_DATA_SENT_4.json", candidate)

In [9]:
create_popularity_json(candidate, "ALL_DATA_SENT_4.json", "POPULARITY.json")

16
['13', '04', '2010']
['06', '04', '2010']
['20', '05', '2010']
['18', '03', '2016']
['29', '11', '2017']
['05', '04', '2016']
['06', '04', '2016']
['06', '04', '2016']
['07', '04', '2016']
['17', '09', '2010']
['13', '11', '2014']
['18', '03', '2009']
['4', '3', '2018']
['17', '2', '2018']
['06', '12', '2011']
['08', '04', '2016']
4
['19', '8', '2016']
['10', '5', '2016']
['6', '5', '2016']
['28', '11', '2016']
10
['14', '05', '2014']
['01', '01', '2015']
['9', '4', '2018']
['05', '06', '2014']
['01', '10', '2014']
['4', '3', '2018']
['9', '4', '2018']
['9', '4', '2018']
['23', '12', '2015']
['04', '06', '2014']
30
['02', '03', '2017']
['6', '3', '2018']
['18', '12', '2013']
['13', '01', '2017']
['26', '02', '2014']
['04', '11', '2015']
['18', '12', '2017']
['26', '08', '2016']
['19', '1', '2018']
['22', '1', '2018']
['22', '1', '2018']
['2', '4', '2018']
['26', '10', '2014']
['30', '03', '2017']
['14', '11', '2014']
['16', '1', '2018']
['9', '4', '2018']
['22', '1', '2018']
['3', '

In [17]:
update_date("ALL_DATA_SENT_4.json", "ALL_DATA_DATE_5.json")

In [4]:
# call script to generate categories and 

all_candidates_str = [x.split() for x in all_candidates]
all_candidates_str = [x.lower() for y in all_candidates_str for x in y]
all_candidates_str.append("ORourke".lower())
extra_words = ["like", "it’s", "uh", "going", "that’s", "think", "actually", "kind", "…", "know", "come", "u", "really"
                       ,"mr", "june", "july", "august", "aug", "festival", "theater", "music", "concert", "dance", "performance"
                       , "jazz", "band", "arts", "art","museum","painting", "artist", "exhibition","gallery", "ms", "sculpture"
                       , "tell", "also","thats", "im", "album", "film", "movie", "opera", "cookbook", "orchestra", "play"
                       , "street", "design", "photograph", "drawing", "collection", "street", "design", "direct", "jan", "may","summer"
                       , "season", "production", "park", "oct", "show", "quartet", "series", "may", "star", "feature", "tonight"
                       , "avenue", "village", "production", "summer", "season", "premiere","ballet", "photograph", "drawing", "boat"
                       , "symphony", "shakespeare", "repertory","pop", "perform", "perform", "night", "classical", "guest", "musical"
                       ,"january", "jan", "february", "feb", "march", "mar", "april", "apr","may", "june", "july", "august", "aug"
                       ,"september", "sept", "october", "oct", "november", "nov", "december", "dec", "literature"]

extra_words.extend(all_candidates_str)

## the following is a script that reads the most recent all_data json,  
## and produces a json with 20 top words
find_top_20(extra_words, candidate, "ALL_DATA_DATE_5.json", "TOP_20_WORDS.json", "TOP_20_VALS.json")

In [5]:
with open("candidate.json", 'w') as f:
        f.write(candidate.to_json(orient = "records"))

In [6]:
candidate = add_sentiment_to_candidate("ALL_DATA_DATE_5.json", "candidate.json", "candidate_sent_1.json")

In [ ]:
### TO CSV 

In [7]:
popularity_df = pd.read_json("POPULARITY.json")
popularity_df = popularity_df[['april', 'may', 'june', 'july', 'august','september','october','november','december','january',
                        'february','march', 'last_name']]
popularity_df.to_csv("popularity.csv", header=False,index=False)

In [9]:
x = "word"
columns = ["id"]
y = [x+"_"+str(i) for i in range(1,21)]
columns.extend(y)
columns.append('last_name')


cloud_df = pd.read_json("TOP_20_WORDS.json")
cloud_df = cloud_df.sort_values(["id"])
cloud_df = reorder_df(cloud_df)
cloud_df = cloud_df[columns]
cloud_df.to_csv("cloud.csv", header=False, index=False)

In [10]:
all_data_raw = pd.read_json("ALL_DATA_DATE_5.json")
all_data_raw.columns
all_data=all_data_raw.rename(index=str, columns={"candidate_fk": "candidate", "state_fk": "state", "source_fk":"source",
                                   "articles_link": "link", "articles_date":"date", "articles_title":"title"})
all_data = all_data.drop(['id', "first_name", "last_name","article_text"],axis=1)
all_data = all_data[["date","link","title","candidate", "source","state", "summary","sentiment_score","issue" ]]

all_data.to_csv("articles.csv", index=False, sep='\t', header=False)

In [13]:
candidate = pd.read_json("candidate_sent_1.json")
candidate = candidate.drop(['id'],axis=1)
candidate = candidate[["name", "state","date_of_birth","place_birth","position", "URL_photo","party", "first_name","last_name", "score_issue_1", "score_issue_2","score_issue_3", "score_issue_4","score_issue_5"]]
candidate.to_csv("candidates.csv",index=False,header=False)

In [ ]:
from sklearn.cluster import KMeans

k = 10
km = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=5,
                verbose=1)
km.fit(tfidf_matrix)
from yellowbrick.text import TSNEVisualizer

tsne = TSNEVisualizer()
tsne.fit(tfidf_matrix)
tsne.poof()

#b = [[2*x for y in a] for x in y]
#c = [[2*x for x in y] for y in a]
#[[int(x) for x in feet] for x in feet]

In [7]:
test_file = open("testfile.txt","w") 
for i in range((len(a))):
    #print(type(a["id"][i]),type(a["articles_link"][i]))
    #print(i, str(a["id"][i]))
    test_file.write(str(a["candidate_fk"][i]) + " " + str(a["id"][i]) + " " + a["articles_link"][i] + '\n')
test_file.close()

In [5]:
from add_manual_articles import*

add_manual("extra_articles.json")

In [6]:
pd.read_json("extra_articles.json")

,article_text,articles_date,articles_link,articles_title,candidate_fk,first_name,id,issue,last_name,newspaper_name,source_fk,state_fk
0,Republican Virginia state Delegate Nick Freita...,16 3 2018,http://www.foxnews.com/opinion/2018/03/16/gop-...,A GOP legislator calls out Democratic hate spe...,9,Nicholas,1,6,Freitas,foxnews,2,2
1,One of the Republican candidates facing Sen. T...,11 4 2018,http://insider.foxnews.com/2018/04/11/corey-st...,GOP Kaine Opponent Rips VA Gov for Vetoing Bil...,9,Nicholas,2,5,Freitas,foxnews,2,2
2,A Republican candidate for Senate in Virginia ...,6 3 2018,http://insider.foxnews.com/2018/03/06/virginia...,'I Won't Accept a False Narrative': VA Lawmake...,9,Nicholas,3,3,Freitas,foxnews,2,2
3,(CNN)How did the Republican Party -- once know...,4 5 2018,https://www.cnn.com/2018/05/04/opinions/strang...,The strange collection of extremists running f...,10,E.W.,4,6,Jackson,CNN,1,2
4,New York (CNN)Donald Trump argued Tuesday that...,22 6 2016,https://www.cnn.com/2016/06/21/politics/donald...,Trump: 'We don't know anything about Hillary i...,10,E.W.,5,6,Jackson,CNN,1,2
5,Washington (CNN)Donald Trump has inflicted a d...,17 5 2016,https://www.cnn.com/2016/05/17/politics/conser...,Conservatives in secretive group 'slow walk' T...,10,E.W.,6,6,Jackson,CNN,1,2
